In [ ]:
def getExistingRelatedItem(item, name):
    items = item.related_items("Service2Service","forward")
    for item in items:
        if item.name == name:
            return item

In [ ]:
#Please enter your username below
YOURUSERNAME = "<Your ArcGIS Online Username>"
feature_service_itemID = "<Feature Service ID Created in Demo0>"

In [ ]:
import arcgis
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import Geometry

In [ ]:
gis = arcgis.gis.GIS("https://www.arcgis.com", YOURUSERNAME)

In [ ]:
item = gis.content.get(feature_service_itemID)
item

In [ ]:
fieldNameCountyObs = "County"
fieldNameCountyBoundary = "JURLBL"

In [ ]:
fset = item.layers[0].query(where='County is not null', out_fields=fieldNameCounty, return_geometry=False, return_distinct_values=True, group_by_fields_for_statistics=fieldNameCounty,out_statistics=[{"statisticType": "count","onStatisticField": fieldNameCountyObs,"outStatisticFieldName": "CountyCount"}])

In [ ]:
fset.df

In [ ]:
flc = FeatureLayerCollection.fromitem(item)
for feature in fset:
    viewName = "Wildlife Observations - " + feature.attributes[fieldNameCounty] + " County"
    existItem = getExistingRelatedItem(item,viewName)
    if existItem is None:
        print("Creating View for " + viewName)
        try:
            existItem = flc.manager.create_view(viewName, spatial_reference=None, extent=None, allow_schema_changes=True, updateable=True, capabilities='Query,Extract', view_layers=None)
        except Exception as e:
            print(e)
    else:
        print("View exists. (" + viewName + ")")
    print("Updating view definition. (" + viewName + ")")
    existItem.layers[0].manager.update_definition({"viewDefinitionQuery":fieldNameCounty + " = '" + feature.attributes[fieldNameCounty] + "'"})
    #result=existItem.share(groups=["Hello Oregon Initiative Open Data Group"])